Importer det vi trenger for geografiske analyser

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

Og det vi trenger for fancy kart

In [ ]:
import hvplot.pandas
from holoviews import opts
import cartopy.crs as ccrs

# støtte for scroll med mus
opts.defaults( 
    opts.Overlay(
        width=900, height=300,
        active_tools=['wheel_zoom']
))

# Last inn rekkverkdata

Hint: 
  - ls lister opp filer. 
  - Først dataframe
    - Pass på tegnsett (encoding='utf-8') og semikolon (sep=';')
  - så dataframe['geometry'] =  dataframe['Kolonne med WKT geometri'].apply(wkt.loads)
  - til sist lager du geodataframe

*NB* For bruk i hvplot må geometrikolonnen hete "geometry"

last inn både rekkverksende og rekkverk, i separate tabeller  

In [ ]:
tmp = pd.read_csv( '14_rekkverksende-eksport.csv', sep=';', encoding='latin1')
tmp['geometry'] =  tmp['Geometri, punkt'].apply(wkt.loads)
rekkverk_ende = gpd.GeoDataFrame( tmp, geometry='geometry' )
rekkverk_ende.crs = { 'init' : 'epsg:25833'}

In [ ]:
tmp = pd.read_csv( '5_rekkverk-eksport.csv', sep=';', encoding='latin1')
tmp['geometry'] =  tmp['Geometri, linje'].apply(wkt.loads)
rekkverk = gpd.GeoDataFrame( tmp, geometry='geometry' )
rekkverk.crs = { 'init' : 'epsg:25833'}

In [ ]:
rekkverk_ende[['vegobjektid', 'startdato', 'Type', 'geometry']]

In [ ]:

# ccrs.UTM(zone=33)
rekkverk.hvplot( crs=ccrs.UTM(zone=33), hover_cols=['vegobjektid', 
            'Rekkverkstype', 'Bruksområde','vegreferanse'], colorbar=False, tiles='OSM', 
            line_width=3)

In [ ]:
plot = rekkverk.hvplot( crs=ccrs.UTM(zone=33), hover_cols=['vegobjektid', 
            'Rekkverkstype', 'Bruksområde','vegreferanse'], colorbar=False, tiles='OSM', 
            line_width=3) 

In [ ]:
plot * rekkverk_ende.hvplot(crs=ccrs.UTM(zone=33), hover_cols=['Type', 'vegreferanse'], s=100, color='r' )

In [ ]:
rekkverk_ende

In [ ]:
r_ende = rekkverk_ende[ ['vegobjektid', 'type_id', 'Type', 'vegreferanse', 'geometry' ]].copy()

In [ ]:
rekk = rekkverk[['vegobjektid', 'Rekkverkstype', 'Bruksområde', 'Skinneutrustning', 'vegreferanse', 'geometry']].copy()

In [ ]:
gpd.sjoin( rekk, r_ende, how='inner', op='intersects')

In [ ]:
gpd.sjoin( r_ende, rekk, how='inner', op='intersects')


Merk at join rekkverk, rekkverksende gir to forekomster av samme NVDB objekt _(vegobjektid_left = 671258138)_, ett for hver av rekkverksendene. (To overlapp = to treff). Disse to treffene kan evt aggregeres basert på ID. 